# <p style="padding:10px;background-color:#E28F6B;margin:0;color:white;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 50px;font-family: Verdana; overflow:hidden;font-weight:500">Customer Churn Analysis</p>

<p style="text-align:center; ">
<img src="https://newsdailyarticles.com/wp-content/uploads/2020/09/5-Effective-Tips-To-Reduce-Customer-Churn.jpg" style='width: 500px; height: 350px;'>
</p>


**`Customer churn`** is a major problem for businesses, as it leads to a loss of revenue and customers. Customer churn (or customer attrition) refers to the loss of customers or subscribers for any reason at all. Businesses measure and track churn as a percentage of lost customers compared to the total number of customers over a given time period. This metric is usually tracked monthly and reported at the end of the month. Therefore, it is important to identify the customers who are likely to churn and take appropriate action to retain them. we will use logistic regression to predict customer churn using a telecommunications dataset.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('/kaggle/input/telco-customer-churn-ibm-dataset/Telco_customer_churn.xlsx')
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


# Data Review and Cleaning

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

**Notice: `Total Charges` column has a object data type but it is numerical data, so let's change data type for this column.**

In [4]:
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

In [5]:
# Get the number of columns of each data type
print(f"""Number of:

float features: {len(df.select_dtypes('float').columns)}
int features: {len(df.select_dtypes('int').columns)}
object features: {len(df.select_dtypes('object').columns)}
""")  

Number of:

float features: 4
int features: 6
object features: 23



In [6]:
data_na = (df.isnull().sum() / len(df)) 
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'mean of nulls' :data_na, "number of nulls" : df[data_na.index].isna().sum()})
missing_data

,mean of nulls,number of nulls
Churn Reason,0.734630,5174
Total Charges,0.001562,11


## The reason of nulls and how to handle them?

**Churn Reason column**

About `73%`(5174) of Churn Reason column is missing data, But why?

In [7]:
df['Churn Label'].value_counts(normalize=True)

Churn Label
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

You can see about `73%` of customers have a Churn Label = No, so they don't need to have some Churn Reason.

**We will drop this column becouse a general rule is that, if more than half of the data in a column is missing, it's better to drop.**

**Total Charges column**

In [8]:
df[df['Total Charges'].isna()]

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
2234,4472-LVYGI,1,United States,California,San Bernardino,92408,"34.084909, -117.258107",34.084909,-117.258107,Female,...,Two year,Yes,Bank transfer (automatic),52.55,NaN,No,0,36,2578,NaN
2438,3115-CZMZD,1,United States,California,Independence,93526,"36.869584, -118.189241",36.869584,-118.189241,Male,...,Two year,No,Mailed check,20.25,NaN,No,0,68,5504,NaN
2568,5709-LVOEQ,1,United States,California,San Mateo,94401,"37.590421, -122.306467",37.590421,-122.306467,Female,...,Two year,No,Mailed check,80.85,NaN,No,0,45,2048,NaN
2667,4367-NUYAO,1,United States,California,Cupertino,95014,"37.306612, -122.080621",37.306612,-122.080621,Male,...,Two year,No,Mailed check,25.75,NaN,No,0,48,4950,NaN
2856,1371-DWPAZ,1,United States,California,Redcrest,95569,"40.363446, -123.835041",40.363446,-123.835041,Female,...,Two year,No,Credit card (automatic),56.05,NaN,No,0,30,4740,NaN
4331,7644-OMVMY,1,United States,California,Los Angeles,90029,"34.089953, -118.294824",34.089953,-118.294824,Male,...,Two year,No,Mailed check,19.85,NaN,No,0,53,2019,NaN
4687,3213-VVOLG,1,United States,California,Sun City,92585,"33.739412, -117.173334",33.739412,-117.173334,Male,...,Two year,No,Mailed check,25.35,NaN,No,0,49,2299,NaN
5104,2520-SGTTA,1,United States,California,Ben Lomond,95005,"37.078873, -122.090386",37.078873,-122.090386,Female,...,Two year,No,Mailed check,20.00,NaN,No,0,27,3763,NaN
5719,2923-ARZLG,1,United States,California,La Verne,91750,"34.144703, -117.770299",34.144703,-117.770299,Male,...,One year,Yes,Mailed check,19.70,NaN,No,0,69,4890,NaN
6772,4075-WKNIU,1,United States,California,Bell,90201,"33.970343, -118.171368",33.970343,-118.171368,Female,...,Two year,No,Mailed check,73.35,NaN,No,0,44,2342,NaN


**we have the number of months that the customer uses the service, `Tenure Months`, and `Monthly Charges`.**
**we can replace null values with those calculated by `monthly charges * tenure months`**


In [9]:
df['Total Charges'].fillna(df['Monthly Charges'] * df['Tenure Months'], inplace=True)

## The problem of Low and High Cardinality

* **Low cordinality/Constant Column**: A column that has the same value for every row in a dataset is often considered uninformative and can be a problem in data analysis or machine learning, as it doesn't provide any information.


* **High Cardinality**: When each row in a dataset has a unique value. High cardinality features can present challenges for modeling, as they can lead to a large number of unique categories, which can make the data more difficult to work with.

In [10]:
df.nunique()

CustomerID           7043
Count                   1
Country                 1
State                   1
City                 1129
Zip Code             1652
Lat Long             1652
Latitude             1652
Longitude            1651
Gender                  2
Senior Citizen          2
Partner                 2
Dependents              2
Tenure Months          73
Phone Service           2
Multiple Lines          3
Internet Service        3
Online Security         3
Online Backup           3
Device Protection       3
Tech Support            3
Streaming TV            3
Streaming Movies        3
Contract                3
Paperless Billing       2
Payment Method          4
Monthly Charges      1585
Total Charges        6531
Churn Label             2
Churn Value             2
Churn Score            85
CLTV                 3438
Churn Reason           20
dtype: int64

**The data has many constant features and columns with high cardinality problem.**

Low cordinality columns:
* `Count` column 
* `Country` column 
* `State` column 

High cordinality columns:
* `CustomerID ` column.
* `Zip Code` column.

**We will drop them later.**

In [11]:
df.groupby(['Country','State']).size()

Country        State     
United States  California    7043
dtype: int64

**All of our customers are from the United States of America, California.**

# Data Visualization

In [12]:
Churn_Label_counts= df['Churn Label'].value_counts()
fig2 = px.pie(names= Churn_Label_counts.keys(), values= Churn_Label_counts.values, title='Churn Label Distribution')
fig2.show()

**`26.5%` of customers have stopped using our service, Our task is to know which type of customers are more likely to stop using our service and what actions we can take.**

In [13]:
values= df['Churn Reason'].value_counts(ascending=False).values
keys= df['Churn Reason'].value_counts(ascending=False).keys()

fig = px.bar(x=keys, y=values, color = values, text = values)

fig.update_layout(
    yaxis_title="Churn Reason",
    xaxis_title="Count"
)
fig.show()

**The graph shows that churn reasons can be divided into two main categories:**

* **internal :-** Internal churn reasons are those that are within the company's control, such as **product quality**, **service quality**, and **price.**.

* **external :-** External churn reasons are those that are outside of the company's control, such as **competitor **offerings and **network reliability**.

**Actions that should to take :-**

* The focus should be on **improving product** and **service quality**, and **reducing prices**, in order to address the top three churn reasons.
* It is also important to **improve customer support**, as this is major churn driver.
* The company should **monitor competitor offerings** and **network reliability**, and make adjustments as needed.

## City

In [14]:
df['City'].nunique()

1129

There are `1129` unique cities in dataset.

In [15]:
# Group by 'City' and 'Churn Label' and count the occurrences
grouped = df.groupby(['City', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby('City').transform('sum')
churn_rate = round(grouped / total_counts, 2)
City_ChurnLabel = pd.DataFrame({'churn_rate': churn_rate}).reset_index()

# Create a DataFrame with the total count per city
city_counts = df.groupby('City')['CustomerID'].count().reset_index()
city_counts.rename(columns={'CustomerID': 'count'}, inplace=True)

# Merge the City_ChurnLabel and city_counts DataFrames
City_ChurnLabel = City_ChurnLabel.merge(city_counts, on='City')

# Filter out the rows where 'Churn Label' is 'No' and sort dataframe by count
City_ChurnLabel = City_ChurnLabel[City_ChurnLabel['Churn Label'] == 'Yes']
City_ChurnLabel = City_ChurnLabel.sort_values(by='count', ascending=False)
City_ChurnLabel.drop(columns='Churn Label', inplace=True)

City_ChurnLabel.head()

,City,churn_rate,count
976,Los Angeles,0.30,305
1502,San Diego,0.33,150
1518,San Jose,0.26,112
1479,Sacramento,0.24,108
1508,San Francisco,0.30,104


In [16]:
fig = px.bar(City_ChurnLabel.head(20), x='City', y= 'count', title='City Counts', text ='count')
fig.update_layout(
    xaxis_title="Name of city",
    yaxis_title="Count"
)
fig.show()

**We see the largest number of customers in the Los Angeles counted `305`, then San Diego counted `150`.**

In [17]:
fig = px.bar(City_ChurnLabel.sort_values(by='churn_rate', ascending=True).head(30),
             x='City', y= 'churn_rate', title='churn rate by city', text ='churn_rate', color='churn_rate')
fig.update_layout(
    xaxis_title="Name of city",
    yaxis_title="churn rate"
)
fig.show()

In [18]:
fig = px.bar(City_ChurnLabel.sort_values(by='churn_rate', ascending=False).head(30),
             x='City', y= 'churn_rate', title='churn rate by city', text ='churn_rate', color='churn_rate')
fig.update_layout(
    xaxis_title="Name of city",
    yaxis_title="churn rate"
)
fig.show()

**There are 12 city have no churned costumers**

In [19]:
fig = px.scatter(City_ChurnLabel, x="count",
                 y="churn_rate", 
                 size = 'churn_rate', labels='City',
                 color = 'churn_rate')
fig.show()

**We observe the churn rate above `40%` only, where we had a small number of customers.**

## Latitude and Longitude

In [20]:
# Group by Latitude', 'Longitude' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Latitude','Longitude', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby(['Latitude','Longitude']).transform('sum')
churn_rate = round(grouped / total_counts, 2)
loc_ChurnLabel = pd.DataFrame({'churn_rate': churn_rate}).reset_index()

# Create a DataFrame with the total count 
loc_counts = df.groupby(['Latitude','Longitude'])['CustomerID'].count().reset_index()
loc_counts.rename(columns={'CustomerID': 'count'}, inplace=True)

# Merge the City_ChurnLabel and city_counts DataFrames
loc_ChurnLabel = loc_ChurnLabel.merge(loc_counts, on=['Latitude','Longitude'])

# Filter out the rows where 'Churn Label' is 'No' and sort dataframe by count
loc_ChurnLabel = loc_ChurnLabel[loc_ChurnLabel['Churn Label'] == 'Yes']
loc_ChurnLabel = loc_ChurnLabel.sort_values(by='count', ascending=False)
loc_ChurnLabel.drop(columns='Churn Label', inplace=True)

loc_ChurnLabel.head()

,Latitude,Longitude,churn_rate,count
1,32.555828,-117.040073,0.2,5
672,34.038983,-117.991372,0.6,5
698,34.050198,-118.210946,0.2,5
696,34.049841,-118.338460,0.2,5
690,34.048013,-118.293953,0.2,5


In [21]:
fig = px.scatter_mapbox(loc_ChurnLabel,
                        lat="Latitude", lon='Longitude',
                        hover_data= ['count'], mapbox_style='open-street-map',
                        color="count"
        )
fig.show()

In [22]:
fig = px.scatter_mapbox(loc_ChurnLabel,
                        lat="Latitude", lon='Longitude',
                        hover_data= ['churn_rate'], mapbox_style='open-street-map',
                        color="churn_rate"
        )
fig.show()

## Tenure Months

In [23]:
fig = px.histogram(df, x="Tenure Months", color="Churn Label" )
fig.show()

**The number of clients in the churn ceases to decline sharply after 5 months in the service.**

In [24]:
TenureMonths= df.groupby('Churn Label')['Tenure Months'].quantile([.50,.75,.90,.95]).reset_index()
TenureMonths['level_1']=TenureMonths['level_1'].astype(str)

fig = px.bar(TenureMonths, x='level_1', y= 'Tenure Months',
             color= 'Churn Label', barmode="group", text = 'Tenure Months' )

fig.update_xaxes(title="Quantiles")
fig.show()

**`50%` of the customers who left the service did so in the first 10 months.**

## Contract type

In [25]:
Contract_Label_counts= df['Contract'].value_counts()
fig1 = px.pie(names= Contract_Label_counts.keys(), values= Contract_Label_counts.values, title='Contract Distribution')
fig1.show()

In [26]:
# Group by 'Contract' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Contract', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby('Contract').transform('sum')
normalized_count = round(grouped / total_counts,2)
Contract_ChurnLabel = pd.DataFrame({'normalized_count': normalized_count}).reset_index()

Contract_ChurnLabel

,Contract,Churn Label,normalized_count
0,Month-to-month,No,0.57
1,Month-to-month,Yes,0.43
2,One year,No,0.89
3,One year,Yes,0.11
4,Two year,No,0.97
5,Two year,Yes,0.03


In [27]:
fig = px.bar(Contract_ChurnLabel, x='normalized_count', y='Contract', color='Churn Label',
             title='100% Stacked Bar Chart by Contract and Churn Label', text= 'normalized_count',
             labels={'normalized_count': 'Percentage'})

fig.update_layout(barmode='relative')  # Make the bar chart 100% stacked

fig.show()

**Churn rate is highest for month-to-month contracts at `43%` that's mean `43%` of customers who had a Month-to-month contract type left the service. The churn rate is lower for one-year contracts, at `11%`, and even lower for two-year contracts, at `3%`.**


In [28]:
fig = px.pie(df.groupby(['Contract','Churn Label'])['CustomerID'].count().reset_index(), 
             values='CustomerID', 
            names='Contract',
            facet_col = 'Churn Label',
            title = 'Churn rate by contract type')
            
fig.show()

**`88.7%` of customers who left the service had a Month-to-month contract type and only `2.57%`  of customers who left the service had a two-year contract**

## Gender

In [29]:
df['Gender'].value_counts(normalize=True)

Gender
Male      0.504756
Female    0.495244
Name: proportion, dtype: float64

**Gender distribution seems balanced between males and females**

In [30]:
# Group by 'Contract' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Gender', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby('Gender').transform('sum')
normalized_count = round(grouped / total_counts,2)
Gender_ChurnLabel = pd.DataFrame({'rate': normalized_count}).reset_index()

Gender_ChurnLabel

,Gender,Churn Label,rate
0,Female,No,0.73
1,Female,Yes,0.27
2,Male,No,0.74
3,Male,Yes,0.26


In [31]:
fig = px.bar(Gender_ChurnLabel, x='rate', y='Gender', color='Churn Label',
             title='100% Stacked Bar Chart by Contract and Churn Label', text= 'rate',
             labels={'rate': 'Percentage'})

fig.update_layout(barmode='relative')  # Make the bar chart 100% stacked

fig.show()

**We don't see much difference. That's mean the Gender of customer don't related with churn label.**

## Payment Method

In [32]:
df['Payment Method'].value_counts()

Payment Method
Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: count, dtype: int64

In [33]:
Payment_Method_counts= df['Payment Method'].value_counts()
fig = px.pie(names= Payment_Method_counts.keys(), values= Payment_Method_counts.values, title='Payment Method Distribution')
fig.show()

**Most of customers use electronic check as a payment method about `33.6%`**

In [34]:
# Group by 'Contract' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Payment Method', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby('Payment Method').transform('sum')
normalized_count = round(grouped / total_counts,2)
Payment_Method_ChurnLabel = pd.DataFrame({'rate': normalized_count}).reset_index()

fig = px.bar(Payment_Method_ChurnLabel, x='rate', y='Payment Method', color='Churn Label',
             title='100% Stacked Bar Chart by Payment Method and Churn Label', text= 'rate',
             labels={'rate': 'Percentage'})

fig.update_layout(barmode='relative')  # Make the bar chart 100% stacked

fig.show()

**It looks like for customers with an electronic check as a payment method with 45% churn rate.** 
**Credit card payment method has the least churn rate about `15%`**

## Internet Service

In [35]:
df['Internet Service'].value_counts()

Internet Service
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64

In [36]:
# Group by 'Internet Service' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Internet Service', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby('Internet Service').transform('sum')
normalized_count = round(grouped / total_counts,2)
Internet_Service_ChurnLabel = pd.DataFrame({'rate': normalized_count}).reset_index()

fig = px.bar(Internet_Service_ChurnLabel, x='rate', y='Internet Service', color='Churn Label',
             title='100% Stacked Bar Chart by Internet Service and Churn Label', text= 'rate',
             labels={'rate': 'Percentage'})

fig.update_layout(barmode='relative')  # Make the bar chart 100% stacked

fig.show()

**Customers with optical fiber Internet have the highest churn rate about `42%` and the least churn rate by clients with no internet service about `7%`**

## Payment Method and Internet Service

In [37]:
# Group by 'Payment Method','Internet Service' and 'Churn Label' and count the occurrences
grouped = df.groupby(['Payment Method','Internet Service', 'Churn Label'])['CustomerID'].count()
total_counts = grouped.groupby(['Payment Method','Internet Service']).transform('sum')
churn_rate = round(grouped / total_counts, 2)
Payment_InternetService_ChurnLabel = pd.DataFrame({'churn_rate': churn_rate}).reset_index()

# Create a DataFrame with the total count per city
Payment_InternetService_counts = df.groupby(['Payment Method','Internet Service'])['CustomerID'].count().reset_index()
Payment_InternetService_counts.rename(columns={'CustomerID': 'count'}, inplace=True)

# Merge the City_ChurnLabel and city_counts DataFrames
Payment_InternetService_ChurnLabel = Payment_InternetService_ChurnLabel.merge(Payment_InternetService_counts, on=['Payment Method','Internet Service'])

# Filter out the rows where 'Churn Label' is 'No' and sort dataframe by count
Payment_InternetService_ChurnLabel = Payment_InternetService_ChurnLabel[Payment_InternetService_ChurnLabel['Churn Label'] == 'Yes']
Payment_InternetService_ChurnLabel = Payment_InternetService_ChurnLabel.sort_values(by='count', ascending=False)
Payment_InternetService_ChurnLabel.drop(columns='Churn Label', inplace=True)

Payment_InternetService_ChurnLabel

,Payment Method,Internet Service,churn_rate,count
15,Electronic check,Fiber optic,0.53,1595
23,Mailed check,No,0.10,741
13,Electronic check,DSL,0.32,648
3,Bank transfer (automatic),Fiber optic,0.29,646
19,Mailed check,DSL,0.21,613
9,Credit card (automatic),Fiber optic,0.25,597
7,Credit card (automatic),DSL,0.12,594
1,Bank transfer (automatic),DSL,0.09,566
5,Bank transfer (automatic),No,0.05,332
11,Credit card (automatic),No,0.03,331


In [38]:
#Creating Sunburst chart based on air, ground or naval unit type
fig = px.sunburst(Payment_InternetService_ChurnLabel, path=['Internet Service', 'Payment Method'], values='count',
                  color='Internet Service', 
                  title="Count of customer by Payment Method and Internet Service")

fig.show()

**Among the customers with optical fiber Internet, most of the customers used an electronic receipt as a means of payment.**

In [39]:
fig = px.bar(Payment_InternetService_ChurnLabel, x='Payment Method', y='count', color='Internet Service',
             barmode="group", text='count'
            )
fig.show()

**Most of customers who used electronic check Payment Method with fiber Internet counted `1595`  but only `122` customers with no internet service**

In [40]:
#Creating Sunburst chart based on air, ground or naval unit type
fig = px.sunburst(Payment_InternetService_ChurnLabel, path=['Internet Service', 'Payment Method'], values='churn_rate',
                  color='Internet Service', 
                  title="churn rate of customer by Payment Method and Internet Service")

fig.show()

In [41]:
fig = px.bar(Payment_InternetService_ChurnLabel, x='Payment Method', y='churn_rate', color='Internet Service',
             barmode="group", text='churn_rate'
            )
fig.show()

**For all payment methods especially for electronic and mailed checks, fiber optic internet with highest churn rates opposite no internet service** 

In [42]:
fig = px.histogram(df, x="Total Charges", color="Churn Label", marginal="box")
fig.show()

**The median charges of churned customers are lower than the median charges of non-churned customers more than 2 times.**

**Total charges go downhill sharply**

## Tech Support

In [43]:
grouped = df.groupby(['Tech Support', 'Churn Label'])['CustomerID'].count()
fig = px.pie(grouped.reset_index(), values='CustomerID', facet_col = 'Churn Label',
             names='Tech Support',
             title = 'Tech support option and churn')
fig.show()

**`77,4%` of the customers who left the service did not have the tech support option enabled.**

In [44]:
total_counts = grouped.groupby('Tech Support').transform('sum')
normalized_count = round(grouped / total_counts,2)
Internet_Service_ChurnLabel = pd.DataFrame({'rate': normalized_count}).reset_index()

fig = px.bar(Internet_Service_ChurnLabel, x='rate', y='Tech Support', color='Churn Label',
             title='100% Stacked Bar Chart by Tech Support and Churn Label', text= 'rate',
             labels={'rate': 'Percentage'})

fig.update_layout(barmode='relative')  # Make the bar chart 100% stacked

fig.show()

**Costumers don't have Tech Support have the highest churn rate about `42%`** 

# Conclusion

# UPVOTE if you find it good analysis and visualization 